# **NOTEBOOK TEMPORAIRE DE CLASSEMENT ET CALCUL DES RESULTATS ATTENDUS PAR BM**

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Lin_uniq_BdxM\Lin_uniq_BdxM\Traitements')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Polygon, box
from shapely.ops import nearest_points
import re
from copy import copy, deepcopy

pd.set_option('display.max_columns', 100)

Idée générale : 
affecter chaque troncon à une commune pour l'agregation par commune, puis chaque troncon à un secteur pour l'agreg par secteur.
Ensuite on calcul le veh_km en se basant sur l'attribut "groupe" qui separent les troncons utilisés pour le calcul de la longueur ou non

Données Necessaires : 
- le fichier de trafic linearise
- le fichier des communes
- le fichier des secteurs

In [404]:
#import des données de base : voies cat1234 linéarisées et fichiers de communes
trafic_lin=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\gdf_rhv_trafic_1234_cpt.shp')
communes=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\communes_BdxM_2.shp',encoding='UTF-8')
communes['geom_c']=communes.geometry
secteurs=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\zones\grands_secteurs.shp',encoding='UTF-8')
secteurs['geom_s']=secteurs.geometry
secteurs.rename(columns={'nom':'nom_sect'},inplace=True)

In [405]:
#on isole les cat_rhv 1,2,3
trafic_cat1213=trafic_lin.loc[trafic_lin['cat_rhv'].isin(['1','2','3','33','61','62','63'])].copy()
# et on n egarde que les lignes d'attribut groupe == 1 (qui servent aux calculs de la longueur
trafic_cat1213_grp1=trafic_cat1213.loc[trafic_cat1213['groupe']==1].copy()
trafic_cat1213_grp1['longueur']=trafic_cat1213_grp1.geometry.length/1000

In [407]:
#jointure entre les lignes et les communes
joint_lin_com=gp.sjoin(trafic_cat1213_grp1[['ident','geometry','cat_rhv','groupe','numero', 'tmjo_2_sen','type_cpt','id_cpt_exp','longueur']], communes[['INSEE_COM','NOM','geometry', 'geom_c']],how='left',op='intersects')

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [408]:
#calcul des longueurs et pourcentage de recouvrement
joint_lin_com['lg_intersect']=joint_lin_com.apply(lambda x : x['geometry'].intersection(x['geom_c']).length, axis=1)
joint_lin_com['lg_lgn']=joint_lin_com.geometry.length
joint_lin_com['ratio_lg']=joint_lin_com.lg_intersect/joint_lin_com.lg_lgn
#on ne conserve que le max des recouvrements pour chaque ligne
corresp_ident_commune=joint_lin_com.loc[joint_lin_com.ratio_lg==joint_lin_com.groupby('ident').ratio_lg.transform(max)].copy()

In [409]:
#jointure entre les lignes et les secteurs
joint_lin_sect=gp.sjoin(corresp_ident_commune.drop('index_right',axis=1), secteurs,how='left',op='intersects')

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [410]:
#calcul des longueurs et pourcentage de recouvrement
joint_lin_sect['lg_intersect']=joint_lin_sect.apply(lambda x : x['geometry'].intersection(x['geom_s']).length, axis=1)
joint_lin_sect['lg_lgn']=joint_lin_sect.geometry.length
joint_lin_sect['ratio_lg']=joint_lin_sect.lg_intersect/joint_lin_sect.lg_lgn
#on ne conserve que le max des recouvrements pour chaque ligne
corresp_ident_sect=joint_lin_sect.loc[joint_lin_sect.ratio_lg==joint_lin_sect.groupby('ident').ratio_lg.transform(max)].copy()
corresp_ident_sect.drop_duplicates('ident',inplace=True)

In [411]:
#on ne garde que les attributs qui nous interesse et on calcul le veh_km
pour_calcul=corresp_ident_sect[['geometry','ident','cat_rhv','groupe','numero', 'tmjo_2_sen','type_cpt','id_cpt_exp','longueur','NOM','nom_sect']].copy()
pour_calcul['veh_km']=pour_calcul.tmjo_2_sen*pour_calcul.longueur
pour_calcul['cat_rhv_simplifiee']=pour_calcul.cat_rhv.apply(lambda x : str(int(x)%10))

In [412]:
#modofier attribut_secteur
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Rocade_N-O' in x['numero'], axis=1),'nom_sect']='rocade NO'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Rocade_RD' in x['numero'], axis=1),'nom_sect']='rocade RD'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Rocade_Sud' in x['numero'], axis=1),'nom_sect']='rocade Sud'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('rocade|autoroute a 630|pont d\'aquitaine|pont francois mitterrand',x['numero'].lower()) else False) 
                    and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='rocade bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a 10',x['numero'].lower()) else False) 
                    and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='A10 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a 10',x['numero'].lower()) else False) 
                    and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='A10'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('route nationale 89',x['numero'].lower()) else False) 
                                and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='N89 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('route nationale 89',x['numero'].lower()) else False) 
                                and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='N89'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('autoroute a 63$',x['numero'].lower()) else False) 
                                and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='A63 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('autoroute a 63$',x['numero'].lower()) else False) 
                                and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='A63'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a62',x['numero'].lower()) else False) 
                                and ('bretelle' in x['numero'].lower()), axis=1),'nom_sect']='A62 bretelle'
pour_calcul.loc[pour_calcul.apply(lambda x : (True if re.search('a62',x['numero'].lower()) else False) 
                                and ('bretelle' not in x['numero'].lower()), axis=1),'nom_sect']='A62'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Boulevards_rive_gauche' in x['numero'], axis=1),'nom_sect']='Bld RG'
pour_calcul.loc[pour_calcul.apply(lambda x : 'Grd-sect Boulevards_rive_droite' in x['numero'], axis=1),'nom_sect']='Bld RD'


In [447]:
#stat communales hors reseau Etat ou SCA et Blds
pour_calcul_hors_GI=pour_calcul.loc[pour_calcul.apply(lambda x : True if not re.match('A.|N89|roc|Bld',x['nom_sect']) else False, axis=1)].copy()
stat_commune=pour_calcul_hors_GI.groupby(['NOM','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})

#grandes infras
#stats Blds
blds=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : x[:3]=='Bld')].copy()
stats_blds=blds.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
#stats Rocade et SCA
Rocade_rd_A10_N89=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : x in ('A10','N89','rocade RD'))].copy()
stat_Rocade_rd_A10_N89=Rocade_rd_A10_N89.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
Rocade_sud_A63_A62=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : x in ('A62','A63','rocade Sud'))].copy()
stat_Rocade_sud_A63_A62=Rocade_sud_A63_A62.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
Rocade_NO=pour_calcul.loc[pour_calcul.nom_sect=='rocade NO'].copy()
stat_Rocade_NO=Rocade_NO.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
#pour info
bretelles=pour_calcul.loc[pour_calcul.nom_sect.apply(lambda x : 'bretelle' in x )].copy()
stat_bretelles=bretelles.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})
#concat GI
grandes_infras=pd.concat([stats_blds,stat_Rocade_rd_A10_N89,stat_Rocade_sud_A63_A62,stat_Rocade_NO,stat_bretelles],axis=0, sort=False)

#stats secteurs
stat_secteurs=pour_calcul_hors_GI.groupby(['nom_sect','cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})

#donnees_generales
donnees_generales=pour_calcul.groupby(['cat_rhv_simplifiee' ]).agg({'veh_km' : sum, 'longueur':'sum'})

In [448]:
#export resultats excel
with pd.ExcelWriter(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\cat123\BM_stat_veh-km_cat123_2.xls') as writer:
    donnees_generales.to_excel(writer, sheet_name='Donnees_generales')
    grandes_infras.reset_index().to_excel(writer, sheet_name='Grandes_infras')
    stat_secteurs.reset_index().to_excel(writer, sheet_name='Secteurs')
    stat_commune.reset_index().to_excel(writer, sheet_name='Communes')

In [451]:
stat_secteurs.sum()

veh_km      8.815427e+06
longueur    1.050487e+03
dtype: float64

In [452]:
stat_commune.sum()

veh_km      8.815427e+06
longueur    1.050487e+03
dtype: float64

In [444]:
filaire_source=gp.read_file(r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\Filaire_voie\FV_TRONC_L.shp')
#export fichier complet
trafic_cat1213[['ident','geometry','cat_rhv','groupe','tmjo_2_sen','type_cpt','id_cpt_exp']].merge(
    filaire_source[['IDENT','NOM_VOIE']], left_on='ident', right_on='IDENT').rename(columns=
    {'id_cpt_exp':'id_comptag'}).drop('IDENT', axis=1).to_file(
r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\cat123\FV_cat123_trafic.shp')

In [446]:
#export fichier de calcul longueur
pour_calcul[['ident','geometry','cat_rhv','groupe','tmjo_2_sen','type_cpt','id_cpt_exp','longueur','NOM','nom_sect','veh_km']].merge(
    filaire_source[['IDENT','NOM_VOIE']], left_on='ident', right_on='IDENT').rename(columns=
    {'id_cpt_exp':'id_comptag', 'NOM':'nom_comm'}).drop('IDENT', axis=1).to_file(
r'D:\temp\Linearisation_BM\C19SA0101\C19SA0101\Doc_travail\Donnees_produites\Donnees\resultats\cat123\FV_cat123_grp1_trafic.shp')

In [443]:
trafic_cat1213[['ident','geometry','cat_rhv','groupe','tmjo_2_sen','type_cpt','id_cpt_exp']].merge(
    filaire_source[['IDENT','NOM_VOIE']], left_on='ident', right_on='IDENT').rename(columns=
    {'id_cpt_exp':'id_comptag'})

,ident,geometry,cat_rhv,groupe,tmjo_2_sen,type_cpt,id_comptag,IDENT,NOM_VOIE
0,32024,"LINESTRING Z (417872.270 6420268.390 0.000, 41...",33,1.0,7144.0,ponctuel,889,32024,Cours de l'Yser
1,11343,"LINESTRING Z (417910.910 6420382.350 0.000, 41...",33,1.0,5192.0,ponctuel,888,11343,Cours de l'Yser
2,50064,"LINESTRING Z (419256.300 6422682.180 0.000, 41...",33,1.0,1538.0,ponctuel,1592,50064,Rue de la Rotonde
3,41367,"LINESTRING Z (417920.970 6422756.060 0.000, 41...",33,1.0,3285.0,permanent,Z4CT20,41367,Allée de Bristol
4,5454,"LINESTRING Z (422532.130 6425223.060 0.000, 42...",33,1.0,5162.0,ponctuel,1668,5454,Rue François Villon
5,8734,"LINESTRING Z (420105.060 6425864.250 0.000, 42...",33,1.0,4136.0,permanent,Z25CT22,8734,Rue Joseph Brunet
6,54413,"LINESTRING Z (419459.820 6422844.500 0.000, 41...",33,1.0,504.0,ponctuel,1591,54413,Rue de la Rotonde
7,41265,"LINESTRING Z (417617.020 6422729.730 0.000, 41...",33,1.0,3285.0,permanent,Z4CT20,41265,Place des Quinconces
8,8736,"LINESTRING Z (420170.400 6426013.240 0.000, 42...",33,1.0,2142.0,permanent,Z25CT24,8736,Rue Joseph Brunet
9,18736,"LINESTRING Z (423799.530 6425058.020 0.000, 42...",33,1.0,6209.0,ponctuel,"1198, 1199",18736,Avenue du Peyrou
